In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPool2D, Flatten, Dense, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy, FalsePositives, FalseNegatives, TruePositives, TrueNegatives, Precision, Recall, AUC
import sklearn
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [2]:
dataset, dataset_info = tfds.load('malaria', with_info=True, as_supervised=True, split=['train'], shuffle_files=True)

In [12]:
DATASET_SIZE = len(dataset[0])

In [13]:
TRAIN_RATIO = 0.6
VAL_RATIO = 0.2
TEST_RATIO = 0.2

def splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
  train_dataset = dataset.take(int(TRAIN_RATIO*DATASET_SIZE))

  val_test_dataset = dataset.skip(int(TRAIN_RATIO*DATASET_SIZE))
  val_dataset = val_test_dataset.take(int(VAL_RATIO*DATASET_SIZE))

  test_dataset = val_test_dataset.skip(int(VAL_RATIO*DATASET_SIZE))
  return train_dataset, val_dataset, test_dataset

train_dataset, val_dataset, test_dataset = splits(dataset[0], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

IM_SIZE = 224
def resize_rescale(image, label):
  return tf.image.resize(image, (IM_SIZE, IM_SIZE))/255.0, label

train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)
test_dataset = test_dataset.map(resize_rescale)

BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size=8).batch(BATCH_SIZE)
# test_dataset = test_dataset.shuffle(buffer_size=8).batch(BATCH_SIZE)

In [14]:
class FeatureExtractor(Layer):
  def __init__(self, filters, kernel_size, strides, padding, activation, pool_size):
    super(FeatureExtractor, self).__init__()

    self.conv1 = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, activation=activation)
    self.batch1 = BatchNormalization()
    self.pool1 = MaxPool2D(pool_size=pool_size, strides=strides)

    self.conv2 = Conv2D(filters=filters*2, kernel_size=kernel_size, strides=strides, padding=padding, activation=activation)
    self.batch2 = BatchNormalization()
    self.pool2 = MaxPool2D(pool_size=pool_size, strides=strides)

  def call(self, x, training):
    x = self.conv1(x)
    x = self.batch1(x)
    x = self.pool1(x)

    x = self.conv2(x)
    x = self.batch2(x)
    x = self.pool2(x)

    return x

In [15]:
class LenetModel(Model):
  def __init__(self):
    super(LenetModel, self).__init__()

    self.feature_extractor = FeatureExtractor(8, 3, 1, "valid", "relu", 2)

    self.flatten = Flatten()

    self.dense1 = Dense(100, activation='relu')
    self.batch1 = BatchNormalization()

    self.dense2 = Dense(10, activation='relu')
    self.batch2 = BatchNormalization()

    self.dense3 = Dense(1, activation='sigmoid')

  def call(self, x, training):
    x = self.feature_extractor(x)

    x = self.flatten(x)
    x = self.dense1(x)
    x = self.batch1(x)
    x = self.dense2(x)
    x = self.batch2(x)
    x = self.dense3(x)

    return x

In [16]:
lenet_sub_classed = LenetModel()

In [5]:
lenet_sub_classed(tf.zeros([1,224,224,3])) # build first
lenet_sub_classed.summary()

Model: "lenet_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extractor (FeatureE  multiple                 1488      
 xtractor)                                                       
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  76038500  
                                                                 
 batch_normalization_2 (Batc  multiple                 400       
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             multiple                  1010      
                                                                 
 batch_normalization_3 (Batc  multiple                 

## Classification Metrics

- Negative -> no malaria
- Positive -> malaria
- False Negative: malaria actually, but predicts no malaria (worst scenario in this problem)
- False Positive: no malaria, but predicts malaria
- True Negative / True Positive

- Precision = TP/(TP+FP)
- Recall = TP/(TP+FN)
- Accuracy = (TN+TP)/(TN+TP+FN+FP)
- F1-score = 2PR/(P+R)

We want to lower the FN, maximize the recall.

Accuracy does not give priority for FN or FP.

ROC Plot

1. Modify the threshold -> If we reduce the threshold for malaria, predicts malaria more. Reduces FN.


In [17]:
metrics = [TruePositives(name='tp'),
           FalsePositives(name='fp'),
           TrueNegatives(name='tn'),
           FalseNegatives(name='fn'),
           BinaryAccuracy(name='accuracy'),
           Precision(name='precision'),
           Recall(name='recall'),
           AUC(name='auc')]

In [18]:
lenet_sub_classed = tf.keras.models.load_model('lenet')

In [9]:
lenet_sub_classed.compile(optimizer=Adam(learning_rate=0.001),
              loss=BinaryCrossentropy(),
              metrics=metrics)
history = lenet_sub_classed.fit(train_dataset, validation_data=val_dataset, epochs=1, verbose=1)

517/517 [==============================] - 71s 106ms/step - loss: 0.4816 - tp: 6383.0000 - fp: 1945.0000 - tn: 6327.0000 - fn: 1879.0000 - accuracy: 0.7687 - precision: 0.7665 - recall: 0.7726 - auc: 0.8496 - val_loss: 0.6195 - val_tp: 2261.0000 - val_fp: 920.0000 - val_tn: 1828.0000 - val_fn: 502.0000 - val_accuracy: 0.7420 - val_precision: 0.7108 - val_recall: 0.8183 - val_auc: 0.8545


In [19]:
test_dataset_ = test_dataset.batch(1)

In [20]:
lenet_sub_classed.evaluate(test_dataset_)

5513/5513 [==============================] - 54s 7ms/step - loss: 0.6547 - tp: 2186.0000 - fp: 928.0000 - tn: 1829.0000 - fn: 570.0000 - accuracy: 0.7283 - precision: 0.7020 - recall: 0.7932 - auc: 0.8370


[0.6547425389289856,
 2186.0,
 928.0,
 1829.0,
 570.0,
 0.728278636932373,
 0.7019910216331482,
 0.7931784987449646,
 0.8370429277420044]

In [23]:
# lenet_sub_classed.save('lenet')

### Visualizing Confusion matrix

In [22]:
test_dataset_

<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [23]:
labels = []
inp = []
for x,y in test_dataset.as_numpy_iterator():
  labels.append(y)
  inp.append(x)

In [24]:
inp[0].shape

(224, 224, 3)

In [11]:
np.array(inp).shape

(5513, 1, 224, 224, 3)

In [12]:
np.array(inp)[:, 0, ...].shape

(5513, 224, 224, 3)

In [ ]:
predicted = lenet_sub_classed.predict(np.array(inp)[:, 0, ...])
print(predicted.shape)

In [21]:
predicted[:,0]

array([0.07523643], dtype=float32)

In [20]:
labels

[array([0, 0])]

In [ ]:
threshold = 0.5

cm = confusion_matrix(labels[0], predicted > threshold)
print(cm)
plt.figure(figsize=(8,8))

sns.heatmap(cm, annot=True,)
plt.title('Confusion matrix - {}'.format(threshold))
plt.ylabel('Actual')
plt.xlabel('Predicted')